In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy import random
import logging

logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
except RuntimeError as e:
    logging.info(e)

In [4]:
dnn = tf.keras.models.load_model('./dnn/1.h5')

In [5]:
data = np.load('../../Data/sample_NuFit0911.npz')
data_all = np.column_stack([data['ve_dune'][:,:36], data['vu_dune'][:,:36], data['vebar_dune'][:,:36], data['vubar_dune'][:,:36]])

In [18]:
#No poisson prediction
IO_prediction = dnn.predict(np.array([data_all[0]]))[0]
NO_prediction = dnn.predict(np.array([data_all[1]]))[0]

IO_delta = np.angle(IO_prediction[2] +1j*IO_prediction[3])
NO_delta = np.angle(NO_prediction[2] +1j*NO_prediction[3])
if IO_delta < 0: IO_delta += 2*np.pi
if NO_delta < 0: NO_delta += 2*np.pi
IO_prediction = np.array([IO_prediction[0], IO_prediction[1], IO_delta])*180/np.pi
NO_prediction = np.array([NO_prediction[0], NO_prediction[1], NO_delta])*180/np.pi

print('IO prediction = ', IO_prediction)
print('NO prediction = ', NO_prediction)

IO prediction =  [  8.54249796  49.07385051 193.47603755]
NO prediction =  [  8.44700423  48.81821702 286.99574113]
